<a href="https://colab.research.google.com/github/royam0820/prompt-engineering/blob/main/l3_iterative_prompt_development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Iterative Prompt Development
In this lesson, you'll iteratively analyze and refine your prompts to generate marketing copy from a product fact sheet.

## Setup

In [ ]:
!pip install openai

In [ ]:
import openai
import os
from google.colab import userdata

In [17]:
from pprint import pprint

In [ ]:
print(openai.__version__)

1.34.0


In [ ]:
openai_api_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_api_key

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

**Note**: In June 2023, OpenAI updated gpt-3.5-turbo. The results you see in the notebook may be slightly different than those in the video. Some of the prompts have also been slightly modified to product the desired results.

## Generate a marketing product description from a product fact sheet

In [10]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture,
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100)
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black,
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities:
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [18]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
pprint(response)


('Introducing our stylish and versatile mid-century inspired office chair, '
 'perfect for both home and business settings. This chair is part of a '
 'beautiful family of office furniture that includes filing cabinets, desks, '
 'bookcases, meeting tables, and more.\n'
 '\n'
 'Customize your chair with several options of shell color and base finishes '
 'to suit your personal style. Choose between plastic back and front '
 'upholstery or full upholstery in a variety of fabric and leather options. '
 'The chair is also available with or without armrests, giving you the '
 'flexibility to create the perfect seating solution for your space.\n'
 '\n'
 'Constructed with a durable 5-wheel plastic coated aluminum base, this chair '
 'features a pneumatic adjust for easy raise/lower action. The dimensions of '
 'the chair are as follows: width 53 cm, depth 51 cm, height 80 cm, seat '
 'height 44 cm, and seat depth 41 cm.\n'
 '\n'
 'Choose between soft or hard-floor caster options and two seat

## Issue 1: The text is too long
- Limit the number of words/sentences/characters.

In [19]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
pprint(response)


('Introducing our versatile and stylish office chair, part of a mid-century '
 'inspired furniture collection. Available in various colors and finishes, '
 'with options for upholstery and armrests. Designed for both home and '
 'business use, with a durable construction and adjustable height feature. '
 'Made in Italy for quality assurance.')


In [13]:
len(response.split())

47

## Issue 2. Text focuses on the wrong details
- Ask it to focus on the aspects that are relevant to the intended audience.

In [20]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

The description is intended for furniture retailers,
so should be technical in nature and focus on the
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
pprint(response)

('Introducing our versatile and stylish office chair, part of a mid-century '
 'inspired furniture collection. Constructed with a durable cast aluminum '
 'shell and base glider coated with modified nylon PA6/PA66. The seat features '
 'high-density HD36 foam for maximum comfort. Made in Italy, perfect for home '
 'or business use.')


In [21]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

The description is intended for furniture retailers,
so should be technical in nature and focus on the
materials the product is constructed from.

At the end of the description, include every 7-character
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
pprint(response)

('Introducing our versatile mid-century office chair, featuring a durable '
 'aluminum base with 5-wheel design and adjustable pneumatic height. Choose '
 'from a variety of shell colors and base finishes, with options for '
 'upholstery in fabric or leather. Perfect for home or business use. Product '
 'ID: SWC-100, SWC-110.')


## Issue 3. Description needs a table of dimensions
- Ask it to extract information and organize it in a table.

In [22]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

The description is intended for furniture retailers,
so should be technical in nature and focus on the
materials the product is constructed from.

At the end of the description, include every 7-character
Product ID in the technical specification.

After the description, include a table that gives the
product's dimensions. The table should have two columns.
In the first column include the name of the dimension.
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website.
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
pprint(response)

('<div>\n'
 '  <p>This mid-century inspired office chair is a stylish and versatile '
 'addition to any workspace. With options for shell color, base finish, and '
 'upholstery type, you can customize this chair to fit your aesthetic '
 'perfectly. The chair is constructed with a durable 5-wheel plastic coated '
 'aluminum base and features a pneumatic adjustment for easy height changes. '
 'Whether you choose the plastic back and front upholstery or the full '
 'upholstery option, this chair is suitable for both home and business '
 'settings. Made in Italy, this chair is not only stylish but also built to '
 'last.</p>\n'
 '  \n'
 '  <p>Product IDs: SWC-100, SWC-110</p>\n'
 '  \n'
 '  <h2>Product Dimensions</h2>\n'
 '  <table>\n'
 '    <tr>\n'
 '      <td>WIDTH</td>\n'
 '      <td>20.87"</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <td>DEPTH</td>\n'
 '      <td>20.08"</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <td>HEIGHT</td>\n'
 '      <td>31.50"</td>\n'
 '    </tr>\n'
 '    <tr>\

## Load Python libraries to view HTML

In [23]:
from IPython.display import display, HTML

In [24]:
display(HTML(response))

WIDTH,"20.87"""
DEPTH,"20.08"""
HEIGHT,"31.50"""
SEAT HEIGHT,"17.32"""
SEAT DEPTH,"16.14"""


## Try experimenting on your own!